# Monte Carlo Calculation of Pi

In [ ]:
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from ipywidgets import Button, IntSlider, HBox, Checkbox, HTML
from matplotlib.animation import FuncAnimation

In [ ]:
Pi = np.pi

samples_inside = [];
samples_outside = [];

pi_x = [];
pi_y = [];

run_button = Button(description='Throw')
clear_button = Button(description='Clear')
num_coin = IntSlider(value=1, min=1, max=200, description='Number of coins:', style={'description_width': 'initial'})
use_quasi = Checkbox(value=False, description='Quasi-Monte Carlo')

label1 = HTML(value = f"<b><font color='blue' size=3>Number of inside coins:</b>")
label2 = HTML(value = f"<b><font color='red' size=3>Number of outside coins:</b>")

In [ ]:
img = plt.figure(tight_layout=True, figsize=(8,4))
img.canvas.header_visible = False

gs = gridspec.GridSpec(5, 2)

ax1 = img.add_subplot(gs[0:5, 0])
ax2 = img.add_subplot(gs[1:4, 1])

x = np.linspace(0, 1, 1000)
ax1.fill_between(x, np.sqrt(1 - x**2), np.zeros(len(x)), alpha=0.4)
ax1.set_xlim(0, 1)
ax1.set_ylim(0, 1)

v1 = ax1.scatter([], [], c="b", s=2)
v2 = ax1.scatter([], [], c="r", s=2)

ax2.axhline(Pi, color='green', label="Exact Pi value")
line_pi, = ax2.plot([], [], 'r-', label="Calculated value")
ax2.set_xlabel("Number of darts")
ax2.set_ylabel("Approximate value of $\pi$")
ax2.legend()

def checkbox_change(c):
    ax1.grid(use_quasi.value)
    if use_quasi.value:
        num_coin.min = 25
        num_coin.max = 200
        num_coin.step = 25
        num_coin.value = 25
    else:
        num_coin.min = 1
        num_coin.max = 200
        num_coin.step = 1
        num_coin.value = 1
        
use_quasi.observe(checkbox_change, names='value')
display(label1, label2)
display(use_quasi)
display(HBox([num_coin, run_button, clear_button]))

In [ ]:
def drop_coin(c):
    if use_quasi.value:
        N = int(num_coin.value/25);
        coin = [];
        for i in np.arange(5):
            for j in np.arange(5):
                x = np.random.uniform(i*0.2, (i+1)*0.2, size=N)
                y = np.random.uniform(j*0.2, (j+1)*0.2, size=N)
                coin.extend(np.array((x, y)).T)
        coin = np.array(coin)
    else:
        coin = np.random.random((num_coin.value, 2))

    samples_inside.extend(coin[(coin**2).sum(axis=1) <= 1.])
    samples_outside.extend(coin[(coin**2).sum(axis=1) > 1.])
    
    x1 = np.shape(samples_inside)[0]
    x2 = np.shape(samples_outside)[0]
    
    pi_x.append(x1 + x2)
    pi_y.append(4.0*x1/(x1+x2))
    
    if samples_inside: 
        v1.set_offsets(samples_inside)
    if samples_outside:
        v2.set_offsets(samples_outside)
        
    line_pi.set_data(pi_x, pi_y)
    ax2.set_xlim(pi_x[0], pi_x[-1])
    ax2.set_title(rf"{pi_x[-1]} samples: $\pi \approx {format(pi_y[-1], '.5f')}$")
    label1.value = rf"<b><font color='blue' size=3>Number of inside coins: {str(x1)}</b>"
    label2.value = rf"<b><font color='red' size=3>Number of outside coins: {str(x2)}</b>"


run_button.on_click(drop_coin)

def clear_figure(c):
    global samples_inside, samples_outside, pi_x, pi_y
    v1.set_offsets([[-1, -1]])
    v2.set_offsets([[-1, -1]])
    line_pi.set_data([],[])
    ax2.set_title('')

    samples_inside = [];
    samples_outside = [];

    pi_x = [];
    pi_y = [];
    
clear_button.on_click(clear_figure)